# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
pip install geomap

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import hvplot.pandas
import requests
import json
import gmaps
import os
import time
import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,21.22,58,96,4.61,GL,1670746090
1,1,anadyr,64.7500,177.4833,10.90,85,100,29.08,RU,1670746314
2,2,ushuaia,-54.8000,-68.3000,42.46,75,75,20.71,AR,1670746315
3,3,albany,42.6001,-73.9662,26.64,76,100,1.01,US,1670746199
4,4,bredasdorp,-34.5322,20.0403,67.95,90,100,16.98,ZA,1670746315


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot

city_data_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.5,
                       xlim=(-180, -30), ylim=(0, 72), tiles='ESRI')



# Display the map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:

cloud_df = city_data_df[city_data_df["Cloudiness"] == 0]


temp_df = cloud_df[(cloud_df["Max Temp"] > 60) & (cloud_df["Max Temp"] < 70)]

wind_df= temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows null values:
ideal_weather_df= wind_df.dropna()
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,sohag,26.5570,31.6948,69.42,24,0,3.44,EG,1670746326
188,188,faya,18.3851,42.4509,61.32,53,0,7.00,SA,1670746343
324,324,chuy,-33.6971,-53.4616,63.43,92,0,6.29,UY,1670746362
330,330,tombouctou,20.0000,-3.0000,69.87,15,0,8.66,ML,1670746363
438,438,jiquinlaca,14.0000,-88.3500,64.17,82,0,4.81,HN,1670746377
466,466,sakakah,29.9697,40.2064,63.70,42,0,5.64,SA,1670746380
501,501,xichang,27.8964,102.2634,63.39,40,0,2.39,CN,1670746385
509,509,riyadh,24.6877,46.7219,62.74,33,0,2.91,SA,1670746283
523,523,pochutla,15.7432,-96.4661,69.82,64,0,3.49,MX,1670746389


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
67,sohag,EG,26.5570,31.6948,24,
188,faya,SA,18.3851,42.4509,53,
324,chuy,UY,-33.6971,-53.4616,92,
330,tombouctou,ML,20.0000,-3.0000,15,
438,jiquinlaca,HN,14.0000,-88.3500,82,
466,sakakah,SA,29.9697,40.2064,42,
501,xichang,CN,27.8964,102.2634,40,
509,riyadh,SA,24.6877,46.7219,33,
523,pochutla,MX,15.7432,-96.4661,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
   
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sohag - nearest hotel: Al Safa Hotel
faya - nearest hotel: No hotel found
chuy - nearest hotel: Alerces
tombouctou - nearest hotel: No hotel found
jiquinlaca - nearest hotel: No hotel found
sakakah - nearest hotel: sakakah residence
xichang - nearest hotel: No hotel found
riyadh - nearest hotel: White Palace Hotel
pochutla - nearest hotel: Hotel El Patio


,City,Country,Lat,Lng,Humidity,Hotel Name
67,sohag,EG,26.5570,31.6948,24,Al Safa Hotel
188,faya,SA,18.3851,42.4509,53,No hotel found
324,chuy,UY,-33.6971,-53.4616,92,Alerces
330,tombouctou,ML,20.0000,-3.0000,15,No hotel found
438,jiquinlaca,HN,14.0000,-88.3500,82,No hotel found
466,sakakah,SA,29.9697,40.2064,42,sakakah residence
501,xichang,CN,27.8964,102.2634,40,No hotel found
509,riyadh,SA,24.6877,46.7219,33,White Palace Hotel
523,pochutla,MX,15.7432,-96.4661,64,Hotel El Patio


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

    
    